# 1.Introduction & Business Problem :
Problem Background:
Today the Business case is represented by Mr.A who wants to open a restaurant in a Business Center close to his home and that he knows well- Val de Fontenay, France. The Business Center size and population is currently increasing  and Mr A thinks there is way of using this to his advantage to open a restaurant. But Mr.A has no experience in the restaurant business and wants to know which type of restaurants he should open.

To sum up : "What restaurant should I open in Val de Fontenay Business Center?"

# 2.Methodology and Data selection
There are multiple ways of answering this question. Let's try to settle down our hypothesis:
- Each Business Center need for restaurant is similar
- The part of covers from each business center for lunch are similar
- The restaurants that will be serving covers to workers are located within a 500-600 m radius from BC center



## Need:In order to know which restaurant should be opened we must know:
1. What is the typical distribution of Restaurant category in Business Centers (Fast food 30%, Pizza 12%, Sushi 3%,..)
2. What is the distribution of the BC : Val de Fontenay
3. Are the restaurants profitable ? Are they Appreciated by the custommers

## Data: 
1. Find a group of Business Center similar to Val de Fontenay in size : [Business Centers List](https://fr.wikipedia.org/wiki/Centre_d%27affaires)
1. Retrieve all restaurants and classify by category for every Business Center 
2. Sort the distribution of Val de Fontenay
3. Use the Like and Rating in order to evaluate the potential success of a category ( a lot of a certain type does not mean it is working properly)

NB: 
As foursquare is applying limitation of 50 rating, it will be impossible to retreive all ratings for all venues.
You will also notice that the amout of like is really poor on all restaurants, we can assume that foursquare is not very popular in france and it is the reason why we lack information. I am not yet certain to use the likes as it might not be accurate due to that. 


In [19]:
!conda install -c conda-forge geopy --yes
print('Geopy installed and imported!')
#!conda install -c conda-forge folium=0.5.0 --yes
#print('Folium installed and imported!')

Solving environment: ...working... done

# All requested packages already installed.

Geopy installed and imported!


In [2]:
import folium
from pandas.io.json import json_normalize
import requests
import pandas as pd

## Data: 
### 1. Long Lat of Business Centers
Retrive Long and Lat of every Business Center using geopy, it will be needed in our research of venues in foursquare


In [3]:
import time
from geopy.geocoders import Nominatim 
address=['Grenoble Europole',
          'Saint-Étienne Châteaucreux',
          'Mériadeck',
          'Innopole FR',
          'Noveos FR',
          'Pleyel Saint Denis',
          "Parc Icade de Rungis",
          "Nanterre Champs Pierreux",
          "Les Bruyères Courbevoie",
          'Boulogne Billancourt Rives de Seine',
          'EuroRennes',
          'Val de Fontenay, FR']

address_m2=[205000,
            250000,
            270000,
            280000,
            250000,
            250000,
            250000,
            250000,
            300000,
            300000,
            300000]
address_dict={x:y for x,y in zip(address, address_m2)}

latitude=[]
longitude=[]
geolocator = Nominatim(user_agent="Jupyter")

def getLatLongList(address_list,latitude,longitude):
    for i in range(len(address_list)):
        lat,lng=getLatLong(address[i])
        latitude.append(lat)
        longitude.append(lng)
    return(latitude,longitude)

def getLatLong(address):
    location = geolocator.geocode(address)
    lat=location.latitude
    lng=location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address,lat, lng))
    return(lat,lng)

getLatLongList(address,latitude,longitude)
address_dict

The geograpical coordinate of Grenoble Europole are 45.1913071, 5.7141017.
The geograpical coordinate of Saint-Étienne Châteaucreux are 45.4433433, 4.3994367.
The geograpical coordinate of Mériadeck are 44.8373835, -0.5856761.
The geograpical coordinate of Innopole FR are 43.5474999, 1.5128101.
The geograpical coordinate of Noveos FR are 48.7776375, 2.2435049239067535.
The geograpical coordinate of Pleyel Saint Denis are 48.9198176, 2.3450792.
The geograpical coordinate of Parc Icade de Rungis are 48.7404828, 2.3529205.
The geograpical coordinate of Nanterre Champs Pierreux are 48.8862592, 2.2140557.
The geograpical coordinate of Les Bruyères Courbevoie are 48.905991, 2.267961.
The geograpical coordinate of Boulogne Billancourt Rives de Seine are 48.8275939, 2.2371424659235677.
The geograpical coordinate of EuroRennes are 48.10130795, -1.6760404045831545.
The geograpical coordinate of Val de Fontenay, FR are 48.8564387, 2.4802895.


{'Grenoble Europole': 205000,
 'Saint-Étienne Châteaucreux': 250000,
 'Mériadeck': 270000,
 'Innopole FR': 280000,
 'Noveos FR': 250000,
 'Pleyel Saint Denis': 250000,
 'Parc Icade de Rungis': 250000,
 'Nanterre Champs Pierreux': 250000,
 'Les Bruyères Courbevoie': 300000,
 'Boulogne Billancourt Rives de Seine': 300000,
 'EuroRennes': 300000}

#### import foursquare credentials

In [16]:
import json
with open('creds.json') as f:
    data = json.load(f)
    CLIENT_ID = data['id']
    CLIENT_SECRET = data['secret']
VERSION = '20200118' # Foursquare API version

## Data : 
### 2. Get all restaurants category 
In order to retrieve more than 50 venues by call for more precision visit [foursquare](https://developer.foursquare.com/docs/resources/categories)

In [24]:
# create the API request URL
url2 = 'https://api.foursquare.com/v2/venues/categories?&client_id={}&client_secret={}&v={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION,
)

# make the GET request
results = requests.get(url2).json()['response']['categories']
df_results = json_normalize(results)

categories = []
i=0
for item in results:
    if(item.get('name')=='Food'):
        for i in range(92):
            if (item['categories'][i] == '4bf58dd8d48988d14c941735'):
                break
            else:
                categories.append(item['categories'][i]['id'])
categories[:5]

['503288ae91d4c4b30a586d67',
 '4bf58dd8d48988d1c8941735',
 '4bf58dd8d48988d14e941735',
 '4bf58dd8d48988d142941735',
 '4bf58dd8d48988d169941735']

## Data:
### 3. Get the Restaurant 
This is our function that retrieves a max of LIMIT=50 venues for each **categories list previously created** in a radius of 500m from [long,lat] position 

In [8]:
def getVenuesCat(names, latitude, longitude,LIMIT=50, radius=500):
    name,lat,lng = names,latitude, longitude
    venues_list=[]    
    venue_list=[]
    for restaurant_category in categories:
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            restaurant_category,
        )

        # make the GET request
        try:
            results = requests.get(url).json()['response']['venues']
            #print(results)
            venues_list.append([(
                name, 
                #lat, 
                #lng,
                v['name'], 
                v['location']['lat'],
                v['location']['lng'], 
                v['categories'][0]['name'],
                v['id']) for v in results])
        except:
             print('No venues of this category')
    
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Business Center', 
                      #'BC Latitude', 
                      #'BC Longitude', 
                      'Venue', 
                      'Venue Latitude', 
                      'Venue Longitude', 
                      'Venue Category',
                    'Venue Id']            
    return(nearby_venues)

## Data:
### 4. Get the like of all venues and store it in a excel .csv file for each Business Center

In [22]:
def get_venues_likes(todo_list,client_id):
    client_iterator=client_id
    count=0
    CLIENT_ID=CLIENT_LIST[client_iterator]
    CLIENT_SECRET=SECRET_LIST[client_iterator]
    for v in todo_list:
        url_likes = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(v, CLIENT_ID, CLIENT_SECRET, VERSION)
        result_likes = requests.get(url_likes).json()
        #print("\n count: {}  client ID: {}".format(count,CLIENT_ID))
        count=count+1
        if(count%450==449):
            client_iterator=+1
            CLIENT_ID=CLIENT_LIST[client_iterator]
            CLIENT_SECRET=SECRET_LIST[client_iterator]  
        
        try:
            likes.append(result_likes['response']['likes']['count'])
            #print(result3['response']['likes']['count'])
        except:
            likes.append('NaN')
            
    print("Client iterator:{}\n Count iterator {}".format(client_iterator,count))

def get_all_likes(address_list,client_id_iterator=0):
    i=client_id_iterator
    for address_items in address_list:
        likes[:]=[]
        lat,long=getLatLong(address_items)
        try:
            df_temp=getVenuesCat(address_items,lat,long,radius=500)
            get_venues_likes(df_temp['Venue Id'],i)
            df_temp['Venue Likes']=likes
            df_temp.to_csv("business center venues/venues{}.csv".format(address_items),encoding='utf-8', index=False)
            #drop from the address_list : address_likes_todo -> allow to see the status of the 
            #address_list.drop(address)
        except:
            print('an error occured at address{}\n Like_list : {}\n Dataframe : df_temp').format(address_items,likes,df_temp)


## Data : Collection completed
Next 3 lines complete the data collection part you can **see the results in the git repo**. See you soon for Data Analysis

In [21]:
likes=[]
address_todo=address
get_all_likes(address_todo)

The geograpical coordinate of Mériadeck are 44.8373835, -0.5856761.


This work is working but limitation of API led me to exclude it 

In [103]:
def get_venues_ratings(todo_list,client_id):
    client_iterator=client_id
    count=0
    CLIENT_ID=CLIENT_LIST[client_iterator]
    CLIENT_SECRET=SECRET_LIST[client_iterator]
    for v in todo_list:
        url_ratings = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(v, CLIENT_ID, CLIENT_SECRET, VERSION)
        result_ratings = requests.get(url_ratings).json()
        #print("\n count: {}  client ID: {}".format(count,CLIENT_ID))
        count=count+1
        if(count%50==49):
            client_iterator+=1
            CLIENT_ID=CLIENT_LIST[client_iterator]
            CLIENT_SECRET=SECRET_LIST[client_iterator]           
        
        try:
            ratings.append(result_ratings['response']['venue']['rating'])
            #print(result2['response']['venue']['rating'])
        except:
            ratings.append('No ratings')
        try:
            prices.append(result_ratings['response']['venue']['price']['tier'])
        except:    
            prices.append('0')#0 for no information 
            #print('This venue has not been rated yet. WHY? {}'.format(result2))
    print("Client iterator:{}\n Count iterator {}".format(client_iterator,count))

def get_all_ratings(address_list,client_id_iterator=0):
    i=client_id_iterator
    for address_items in address_list:
        ratings[:]=[]
        prices[:]=[]
        i=i+1
        lat,long=getLatLong(address_items)
        try:
            df_temp=pd.DataFrame()
            df_temp=pd.read_csv("business center venues/venues{}.csv".format(address_items),encoding='utf-8')
            get_venues_ratings(df_temp['Venue Id'],i)
            print(ratings)
            print(df_temp)
            df_temp['Venue Ratings']=ratings
            df_temp.to_csv("business center venues/venues{}.csv".format(address_items),encoding='utf-8', index=False)
            #drop from the address_list : address_likes_todo -> allow to see the status of the 
            #address_list.drop(address)
        except:
            print('an error occured at address{}\n Ratings_list : {}\n Dataframe : df_temp').format(address_items,ratings)
                

In [104]:
ratings=[]
prices=[]
get_all_ratings(address_ratings)